In [1]:
## packages used
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import scanpy as sc
import spaTrack as spt

In [ ]:
# package versions: matplotlib 3.9.4; numpy 1.24.4; pandas 1.4.3; 
# package versions: scanpy 1.9.8; spaTrack 1.0.2; python 3.9

In [ ]:
# Step 1 Preparation: in Python make anndata object for spaTrack analysis

In [ ]:
# Load TSV file with gene expression data, working directory is /home/gd2/
adata = sc.read("/home/gd2/Sample_TransposedGeneExpression.tsv")

In [ ]:
# Look at dimensions of adata, confirm correct
adata

In [ ]:
# Load TSV file with cell type information and spatial coordinates
df_annot = pd.read_table("/home/gd2/Sample_TissueCoords.tsv")

In [ ]:
# Look at table in variable df_annot
df_annot

In [ ]:
# Input cell type information from df_annot into adata.obs[‘cluster’]
adata.obs["cluster"] = df_annot["cluster"].values

In [ ]:
# Input spatial coordinates from df_annot into adata.obsm[“X_spatial”]
adata.obsm["X_spatial"] = df_annot[["x", "y"]].values

In [ ]:
# Run this step
adata.layers["counts"] = adata.X

In [ ]:
# Perform filtration and normalization steps
sc.pp.filter_genes(adata, min_cells=0)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

In [ ]:
# View figure of cell types on spatial plot, save a png file
fig, axs = plt.subplots(figsize=(7,7))
ax = sc.pl.embedding(adata, basis = 'X_spatial', show = False, color = 'cluster', ax=axs, frameon=False, title = ' ' , palette = 'tab20', size=100, save = 'Sample.png’)

In [ ]:
# Save anndata object and use for spaTrack analysis
adata.write('/home/gd2/Sample_Anndata.h5ad')

In [ ]:
# Step 2 spaTrack Analysis
# Read in the anndata file
adata = sc.read_h5ad('/home/gd2/Sample_Anndata.h5ad')

In [ ]:
# Look at adata contents
adata

In [ ]:
# Look at entropy value for each cell type to help identify starting cluster
adata = spt.assess_start_cluster(adata)

In [ ]:
# Generate a figure with entropy values for each cell type
spt.assess_start_cluster_plot(adata)

In [ ]:
# Select starting cells based on cell type, start cells with low entropies
start_cells=spt.set_start_cells(adata,select_way='cell_type',cell_type='Tumor_OPC-like_1')
fig, axs = plt.subplots(figsize=(5,5))
plt.scatter(adata.obsm['X_spatial'][:,0],adata.obsm['X_spatial'][:,1],c='#D3D3D3',s=8)
plt.scatter(adata.obsm['X_spatial'][start_cells][:,0],adata.obsm['X_spatial'][start_cells][:,1],c='orange',s=8)

In [ ]:
# Calculate cell transition probability
 adata.obsp["trans"] = spt.get_ot_matrix(adata, data_type="spatial",alpha1=0.5,alpha2=0.5)

In [ ]:
# Calculate cell pseudotime
adata.obs["ptime"] = spt.get_ptime(adata, start_cells)

In [ ]:
 # Calculate vector field velocity
 adata.uns["E_grid"], adata.uns["V_grid"] = spt.get_velocity(adata, basis="spatial", n_neigh_pos=50)

In [ ]:
# Plot pseudotime
fig, axs = plt.subplots(figsize=(5, 5))
sc.pl.embedding(adata, basis='spatial', color='ptime', show=False, ax=axs, color_map='Reds', title='ptime', size=100)
axs.xaxis.set_major_locator(ticker.MultipleLocator(500))
axs.yaxis.set_major_locator(ticker.MultipleLocator(500))

In [ ]:
# Plot quiver plot
fig, axs = plt.subplots(figsize=(5, 5))
sc.pl.embedding(adata, basis='spatial',show=False,title=' ',color='cluster',ax=axs,frameon=False,palette='tab20b',legend_fontweight='normal',alpha=0.1,size=600)                     
axs.quiver(adata.uns['E_grid'][0],adata.uns['E_grid'][1],adata.uns['V_grid'][0],adata.uns['V_grid'][1],scale=0.008)

In [ ]:
# Plot stream plot
fig,axs=plt.subplots(ncols=1,nrows=1,figsize=(6,6))
ax = sc.pl.embedding(adata,  basis='spatial',show=False,title=' ',color='cluster',ax=axs,frameon=False,palette='tab20b',legend_fontweight='normal',alpha=0.8,size=150)
ax.streamplot(adata.uns['E_grid'][0], adata.uns['E_grid'][1], adata.uns['V_grid'][0], adata.uns['V_grid'][1],density=1.8,color='black',linewidth=2.5,arrowsize=1.5)

In [ ]:
# Least Action Path (LAP) step 1
VecFld=spt.VectorField(adata,basis='spatial')

In [ ]:
# Least Action Path (LAP) step 2, LAP_start_point and LAP_end_point are manually entered
fig, ax = plt.subplots(figsize=(6, 6))
                     
LAP_start_point=[6749, 61]
LAP_end_point=[8239,502]
LAP_start_cell=spt.nearest_neighbors(LAP_start_point,adata.obsm['X_spatial'])[0][0]
LAP_end_cell=spt.nearest_neighbors(LAP_end_point,adata.obsm['X_spatial'])[0][0]
                     
plt.scatter(*adata.obsm["X_spatial"].T,c='#D3D3D3',s=15)
plt.scatter(*adata[LAP_start_cell].obsm['X_spatial'].T,s=20)
plt.scatter(*adata[LAP_end_cell].obsm['X_spatial'].T,s=20)

In [ ]:
# Final LAP step
lap=spt.least_action(adata,
                            init_cells=adata.obs_names[LAP_start_cell],
                            target_cells=adata.obs_names[LAP_end_cell],
                            vecfld=VecFld,
                            basis='spatial',
                            adj_key='X_spatial_distances',
                            EM_steps=5,
                            n_points=20
)

In [ ]:
# This step generates Figures S7B,D,F. View optimal transition path
LAP_ptime,LAP_nbrs=spt.map_cell_to_LAP(adata,cell_neighbors=80)
sub_adata=adata[LAP_nbrs,:]
sub_adata.obs['ptime']=LAP_ptime
sub_adata=sub_adata[np.argsort(sub_adata.obs["ptime"].values), :].copy()

fig, ax = plt.subplots(figsize=(6,5))
plt.axis('off')
ax = sc.pl.embedding(adata,  basis='X_spatial',color='cluster',show=False,ax=ax,frameon=False,legend_loc=None,alpha=0.6,size=150)
ax.streamplot(adata.uns['E_grid'][0], adata.uns['E_grid'][1], adata.uns['V_grid'][0], adata.uns['V_grid'][1],density=1.2,color='black',linewidth=2,arrowsize=1.2)
ax = spt.plot_least_action_path(adata,basis='spatial',ax=ax,point_size=120,linewidth=5)
sc.pl.embedding(sub_adata, basis='X_spatial',ax=ax, color="ptime", cmap="YlGnBu",frameon=True,size=150,title=' ')

In [ ]:
# Choose cell type on transition path and filter genes with high variability and expression levels greater 
# than minimum expression proportion
sub_adata_path=sub_adata[sub_adata.obs['cluster'].isin(['Tumor_1'])]
sub_adata_path=spt.filter_gene(sub_adata_path,min_exp_prop=0.1,hvg_gene=3000)

In [ ]:
# Fit gene expression and cell pseudotime using Generalized Additive Model (GAM)
# Step 1
df_res  = spt.ptime_gene_GAM(sub_adata_path,core_number=5)

In [ ]:
#Step 2
df_sig_res = df_res.loc[(df_res['model_fit']>0.05) & (df_res['fdr']<0.05)]

In [ ]:
# Generate heatmap. This step generates Figures S7C,E,G.
sort_exp_sig = spt.order_trajectory_genes(sub_adata,df_sig_res,cell_number=20)                
spt.plot_trajectory_gene_heatmap(sort_exp_sig,smooth_length=100,gene_label_size=15,cmap_name='twilight_shifted')

In [ ]:
# Save anndata object after analysis
adata.write('Sample_Completed_spaTrack_Anndata.h5ad')